In [1]:
import pandas as pd
import qgrid
import json
import os
import functools
import ipywidgets as widgets
from pathlib import Path
from IPython.display import clear_output, display
from ipyleaflet import Map, Icon, Marker, AntPath, basemaps, basemap_to_tiles, LayerException
from OSGridConverter import grid2latlong

import sys
sys.path.append('..')
from src import sandbox, gpx
from src.settings import PROJECT_PATH, DATASET_PATH

In [2]:
file_gen = DATASET_PATH.glob("*walks.json")

dfs = [] # an empty list to store the data frames
for file in file_gen:
    
    with open(file) as f:
        
        data = json.load(f)
        dfs.append(pd.DataFrame(data))

df = pd.concat(dfs, ignore_index=True) # concatenate all the data frames in the list.

display(df)

,Distance,Time,Ascent,Start Grid Ref,Area0,Area1,Name,Rating,Votes,Grade,...,Corbett,Grahams,Sub 2000s,Corbett / Donald,Graham / Donald,Donald,Corbett:,Donalds,Grahams:,Graham:
0,5km / 3 miles,2 - 3 hours,155m (Profile),NG323374,skye,oronsay,"Oronsay Island, Ullinish",3.89,9,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11km / 6.75 miles,NaN,351m (Profile),NG132478,skye,milovaig,Neist and Milovaig clifftop coast walk,3.60,10,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9km / 5.5 miles,NaN,481m (Profile),NG181546,skye,biodanathair,Biod an Athair and Dunvegan Head,3.57,7,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3km / 1.75 miles,2 - 2.5 hours,110m (Profile),NG242482,skye,colbost,"Uiginish Point, near Colbost",2.60,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3km / 1.75 miles,1 - 2 hours,87m (Profile),NG240507,skye,dun-fiadhairt,"Dùn Fiadhairt, near Dunvegan",1.91,11,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1933,10.5km / 6.5 miles,NaN,490m (Profile),NR881369,arran,sail-chalmadale,"Sail Chalmadale, from Dougarie",2.67,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1934,5km / 3 miles,1.5 -2 hours,99m (Profile),NR937506,arran,fairy-dell,"Fairy Dell, Lochranza",3.20,5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1935,15.75km / 9.75 miles,NaN,119m (Profile),NS014466,arran,cock-of-arran,Sannox to Lochranza: Cock of Arran,4.43,7,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1936,16.5km / 10.25 miles,NaN,294m (Profile),NR929507,arran,loch-na-davie,"Loch na Davie circuit, Lochranza",3.44,9,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We have two columns for time as some of the higher graded walks are given in summer conditions. Let's combine those.
Similary, some columns are plurals of others, let's combine those too.

In [3]:
df["Time"] = df["Time"].fillna(df["Time (summer conditions)"])
df = df.drop(columns="Time (summer conditions)")

for col in df.columns:
    if col + 's' in df.columns:
        df[col] = df[col].fillna(df[col + 's'])
        df = df.drop(columns=col + 's')

display(df)

,Distance,Time,Ascent,Start Grid Ref,Area0,Area1,Name,Rating,Votes,Grade,...,Sub 2000,Graham,Munro,Corbett,Corbett / Donald,Graham / Donald,Donald,Corbett:,Grahams:,Graham:
0,5km / 3 miles,2 - 3 hours,155m (Profile),NG323374,skye,oronsay,"Oronsay Island, Ullinish",3.89,9,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11km / 6.75 miles,3 - 5 hours,351m (Profile),NG132478,skye,milovaig,Neist and Milovaig clifftop coast walk,3.60,10,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9km / 5.5 miles,3 - 5 hours,481m (Profile),NG181546,skye,biodanathair,Biod an Athair and Dunvegan Head,3.57,7,3,...,Biod an Athair,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3km / 1.75 miles,2 - 2.5 hours,110m (Profile),NG242482,skye,colbost,"Uiginish Point, near Colbost",2.60,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3km / 1.75 miles,1 - 2 hours,87m (Profile),NG240507,skye,dun-fiadhairt,"Dùn Fiadhairt, near Dunvegan",1.91,11,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1933,10.5km / 6.5 miles,5 - 5.5 hours,490m (Profile),NR881369,arran,sail-chalmadale,"Sail Chalmadale, from Dougarie",2.67,3,3,...,Sail Chalmadale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1934,5km / 3 miles,1.5 -2 hours,99m (Profile),NR937506,arran,fairy-dell,"Fairy Dell, Lochranza",3.20,5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1935,15.75km / 9.75 miles,4.5 - 6 hours,119m (Profile),NS014466,arran,cock-of-arran,Sannox to Lochranza: Cock of Arran,4.43,7,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1936,16.5km / 10.25 miles,5 - 5.5 hours,294m (Profile),NR929507,arran,loch-na-davie,"Loch na Davie circuit, Lochranza",3.44,9,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's have a look at walks that include Munro's only. While we're at it, let's look at the really tough one's! We'll also drop columns that have all null values.

In [4]:
munro_walks = df[(df['Munro'].notnull()) & (df['Grade'].astype(int) >= 3)]
munro_walks = munro_walks.dropna(axis=1, how='all')
display(munro_walks)

,Distance,Time,Ascent,Start Grid Ref,Area0,Area1,Name,Rating,Votes,Grade,Bog,Link,GPX,Graham,Munro,Corbett
49,14km / 8.75 miles,6 - 7 hours,1175m (Profile),NG484296,skye,ambasteir,Am Basteir,3.50,10,5,2,https://www.walkhighlands.co.uk/skye/ambasteir...,https://www.walkhighlands.co.uk/skye/profiles/...,NaN,Am Basteir,NaN
55,9.5km / 6 miles,5 - 6 hours,992m (Profile),NG408206,skye,sgurralasdair,Sgùrr Alasdair,3.52,25,5,2,https://www.walkhighlands.co.uk/skye/sgurralas...,https://www.walkhighlands.co.uk/skye/profiles/...,NaN,Sgùrr Alasdair,NaN
60,9.5km / 6 miles,6 - 8 hours,997m (Profile),NG409225,skye,sgurraghreadaidh,Sgùrr a Mhadaidh and Sgùrr a Ghreadaidh,3.56,9,5,3,https://www.walkhighlands.co.uk/skye/sgurraghr...,https://www.walkhighlands.co.uk/skye/profiles/...,NaN,"Sgùrr a' Ghreadaidh, Sgùrr a' Mhadaidh",NaN
61,13.75km / 8.5 miles,5 - 7 hours,947m (Profile),NG484296,skye,bruachnafrithe,Bruach na Frithe,3.53,15,4,3,https://www.walkhighlands.co.uk/skye/bruachnaf...,https://www.walkhighlands.co.uk/skye/profiles/...,NaN,Bruach na Frithe,NaN
64,15km / 9.25 miles,7 - 11 hours,1159m (Profile),NG408206,skye,sgurrnaneag,Southern Cuillin and Coire Ghrunnda,3.24,17,5,2,https://www.walkhighlands.co.uk/skye/sgurrnane...,https://www.walkhighlands.co.uk/skye/profiles/...,NaN,"Sgùrr nan Eag, Sgùrr Dubh Mòr",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863,28km / 17.5 miles,10 - 13 hours,1727m (Profile),NH216242,lochness,carn-eige,Càrn Eige and Mam Sodhail,3.73,11,4,3,https://www.walkhighlands.co.uk/lochness/carn-...,https://www.walkhighlands.co.uk/lochness/profi...,NaN,"Càrn Eige (or Càrn Eighe), Beinn Fhionnlaidh, ...",NaN
1865,29km / 18 miles,11 - 14 hours,1826m (Profile),NH218315,lochness,mullardoch-munros,The Loch Mullardoch Munros,3.42,12,4,4,https://www.walkhighlands.co.uk/lochness/mulla...,https://www.walkhighlands.co.uk/lochness/profi...,NaN,"Càrn nan Gobhar, Sgùrr na Lapaich, An Riabhach...",NaN
1868,25km / 15.5 miles,7 - 10 hours,1567m (Profile),NH283386,lochness,strathfarrar-munros,Glen Strathfarrar Munros circuit,3.38,24,4,3,https://www.walkhighlands.co.uk/lochness/strat...,https://www.walkhighlands.co.uk/lochness/profi...,NaN,"Sgùrr na Ruaidhe, Càrn nan Gobhar, Sgùrr a' Ch...",NaN
1869,16.5km / 10.25 miles,6 - 8 hours,1096m (Profile),NH216242,lochness,Tollcreagach,"Tom a' Chòinich and Toll Creagach, Glen Affric",3.65,17,4,3,https://www.walkhighlands.co.uk/lochness/Tollc...,https://www.walkhighlands.co.uk/lochness/profi...,NaN,"Toll Creagach, Tom a' Chòinich",NaN


In [5]:
m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(57, 356),
    zoom=6
    )

p = Map(
    basemap=basemap_to_tiles(basemaps.WaymarkedTrails.hiking),
    center=(57, 356),
    zoom=6
    )

#mark = Marker(location=[57, 356], icon=icon)
#m.add_layer(mark)

for p in munro_walks["Start Grid Ref"]:
    loc = grid2latlong(p)
    lat = loc.latitude
    lon = 360 + loc.longitude if loc.longitude < 0 else loc.longitude
    mark = Marker(location=[lat, lon])
    m.add_layer(mark)

m

Map(center=[57, 356], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [10]:
df = sandbox.main()

widget = qgrid.show_grid(df)
widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [8]:
def get_lat_lon_bounds(df):
    
    north = max(df["lat"])
    south = min(df["lat"])
    east = max(df["lon"])
    west = min(df["lon"])
    
    return [(south, west), (north, east)]

def handle_click(map_handle, data, **kwargs):
    
    route = gpx.parse(data["GPX"])
    route = gpx.positive_long(route)
    latlon = list(zip(route.lat, route.lon))
    path = AntPath(locations=latlon)
    
    try:
        map_handle.substitute_layer(handle_click.current, path)
    except (AttributeError, LayerException):
        map_handle.add_layer(path)
        
    handle_click.current = path
    
    map_handle.center = kwargs["coordinates"]
    map_handle.zoom = 11
    # map_handle.fit_bounds = get_lat_lon_bounds(route)
   
    display(data)
    

In [9]:
filtered_df = widget.get_changed_df()

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(57, 356),
    zoom=6
    )

display_max = 100

if len(filtered_df.index) < display_max:
    
    for _, p in filtered_df.iterrows():
        mark = Marker(location=(p["lat"], p["lon"]), draggable=False, title=p["Name"])
        mark.on_click(functools.partial(handle_click, m, p))
        m.add_layer(mark)
        
else:
    print(f"Too many walks to show ({len(filtered_df.index)})! Filter below maximum ({display_max}) ")
    
m
        

Too many walks to show (1936)! Filter below maximum (100) 


Map(center=[57, 356], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…